In [1]:
import pandas as pd
import json
file = open("./resources/216480444-216482440-218407734-T1Disc.csv", "r")

In [2]:
data = [0] * 200
i = 0
for line in file:
    l = line.rstrip('\n')
    l = l.split(',')
    if l[0] == '': continue
    data[i] = l
    i += 1

length 1 itemsets

In [3]:
minSup = 0.3
itemset = {}
for business in data:
    for x in business:
        if x in itemset:
            itemset[x] += 1
        else:
            itemset.update({x: 0})

In [4]:
temp_itemset = {}
for item in itemset:
    if itemset[item] / 200 > minSup:
        temp_itemset.update({item: itemset[item]})
itemset = temp_itemset

length 2 itemsets

In [5]:
temp_itemset = {}
used = []
for item1 in itemset:
    used.append(item1)
    for item2 in itemset:
        if item2 in used: 
            continue
        else: 
            merge = str(item1 + ',' + item2)
            temp_list = merge.split(',')
            for business in data:
                if all(a in business for a in temp_list):
                    if merge in temp_itemset:
                        temp_itemset[merge] += 1
                    else:
                        temp_itemset.update({merge: 0})
itemset = temp_itemset

In [6]:
temp_itemset = {}
for item in itemset:
    if itemset[item] / 200 > minSup:
        temp_itemset.update({item: itemset[item]})
itemset = temp_itemset

length 3 itemsets

In [7]:
def attempt_merge(a, b):
    temp1 = a.split(',')
    temp2 = b.split(',')
    for i in range(len(temp1)-1):
        if temp1[i] != temp2[i]:
            return 0
    ret = ''
    for i in temp1:
        ret += i + ','
    ret += temp2[len(temp2)-1]
    return ret

In [8]:
def gen_candidate(itemset):
    temp_itemset = {}
    used = []
    for item1 in itemset:
        used.append(item1)
        for item2 in itemset:
            if item2 in used: 
                continue
            else: 
                merge = attempt_merge(item1, item2)
                if merge == 0:
                    continue
                temp_list = merge.split(',')
                for business in data:
                    if all(a in business for a in temp_list):
                        if merge in temp_itemset:
                            temp_itemset[merge] += 1
                        else:
                            temp_itemset.update({merge: 0})
    ret_itemset = {}
    for item in temp_itemset:
        if temp_itemset[item] / 200 > minSup:
            ret_itemset.update({item: temp_itemset[item]})
    return ret_itemset

In [9]:
for i in range(2):
    old_itemset = itemset.copy()
    itemset = gen_candidate(itemset)

In [10]:
temp = []
for item in itemset:
    temp.append(itemset[item])
temp.sort()
temp = temp[-10:]

In [11]:
top10 = {}
for t in temp:
    for item in itemset:
        if itemset[item] == t:
            top10.update({item: t})

In [12]:
confidence = {}
for item in top10:
    for old_item in old_itemset:
        if old_item in item:
            confidence.update({item: itemset[item]/old_itemset[old_item]})

In [13]:
temp

[64, 67, 70, 71, 75, 75, 79, 79, 79, 88]

In [14]:
temp = []
for c in confidence:
    temp.append(confidence[c])
temp.sort()
temp = temp[-5:] + temp[:5]

In [15]:
for t in temp:
    for c in confidence: 
        if confidence[c] == t:
            temp_set = c.split(',')
            print('Rule:', str(temp_set[:len(temp_set)-2]), '->', temp_set[len(temp_set)-1] , '| confidence =', round(t, 4))
            break       

Rule: ['is_open', 'RestaurantsTakeOut'] -> BusinessAcceptsCreditCards | confidence = 0.8316
Rule: ['is_open', 'RestaurantsDelivery'] -> RestaurantsGoodForGroups | confidence = 0.8375
Rule: ['RestaurantsTakeOut', 'BusinessAcceptsCreditCards'] -> HasTV | confidence = 0.88
Rule: ['RestaurantsTakeOut', 'RestaurantsDelivery'] -> RestaurantsGoodForGroups | confidence = 0.9375
Rule: ['RestaurantsTakeOut', 'RestaurantsDelivery'] -> HasTV | confidence = 0.9467
Rule: ['is_open', 'RestaurantsTakeOut'] -> RestaurantsGoodForGroups | confidence = 0.7054
Rule: ['is_open', 'RestaurantsTakeOut'] -> HasTV | confidence = 0.7292
Rule: ['is_open', 'RestaurantsTakeOut'] -> HasTV | confidence = 0.7453
Rule: ['is_open', 'BusinessAcceptsCreditCards'] -> HasTV | confidence = 0.75
Rule: ['is_open', 'RestaurantsTakeOut'] -> RestaurantsGoodForGroups | confidence = 0.8101


In [16]:
top10

{'is_open,RestaurantsTakeOut,RestaurantsDelivery,RestaurantsGoodForGroups': 64,
 'is_open,RestaurantsDelivery,BusinessAcceptsCreditCards,RestaurantsGoodForGroups': 67,
 'is_open,RestaurantsTakeOut,RestaurantsGoodForGroups,HasTV': 70,
 'RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards,HasTV': 71,
 'is_open,BusinessAcceptsCreditCards,RestaurantsGoodForGroups,HasTV': 75,
 'RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards,RestaurantsGoodForGroups': 75,
 'is_open,RestaurantsTakeOut,RestaurantsDelivery,BusinessAcceptsCreditCards': 79,
 'is_open,RestaurantsTakeOut,BusinessAcceptsCreditCards,RestaurantsGoodForGroups': 79,
 'is_open,RestaurantsTakeOut,BusinessAcceptsCreditCards,HasTV': 79,
 'RestaurantsTakeOut,BusinessAcceptsCreditCards,RestaurantsGoodForGroups,HasTV': 88}